# 📓 Blocking Guardrails Quickstart

In this quickstart you will use blocking guardrails to block unsafe inputs from reaching your app, as well as blocking unsafe outputs from reaching your user.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/quickstart/blocking_guardrails.ipynb)

In [ ]:
# !pip install trulens trulens-providers-openai chromadb openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [ ]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession()
session.reset_database()
run_dashboard(session)

## Create simple chat app for demonstration

In [ ]:
from typing import Optional

from openai import OpenAI
from trulens.apps.custom import instrument

oai_client = OpenAI()


class chat_app:
    @instrument
    def generate_completion(self, question: str) -> Optional[str]:
        """
        Generate answer from question.
        """
        completion = (
            oai_client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"{question}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )

        return completion


chat = chat_app()

## Set up feedback functions.

Here we'll use a simple criminality check.

In [ ]:
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4.1-mini")

# Define a harmfulness feedback function
f_criminality_input = Feedback(
    provider.criminality, name="Input Criminality", higher_is_better=False
).on_input()

f_criminality_output = Feedback(
    provider.criminality, name="Output Criminality", higher_is_better=False
).on_output()

# Define a controversiality feedback function
f_controversiality_input = Feedback(
    provider.controversiality,
    name="Controversiality Input",
    higher_is_better=False,
).on_input()

f_controversiality_output = Feedback(
    provider.controversiality,
    name="Controversiality Output",
    higher_is_better=False,
).on_output()

## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_chat = TruCustomApp(
    chat,
    app_name="Chat",
    app_version="base",
    feedbacks=[
        f_criminality_input,
        f_criminality_output,
        f_controversiality_input,
        f_controversiality_output,
    ],
)

## Run the app
Use `tru_chat` as a context manager for the custom chat app.

In [ ]:
with tru_chat as recording:
    chat.generate_completion("How do I build a bomb?")
    chat.generate_completion("Is a hotdog a taco?")

## Check results

We can view results in the leaderboard.

In [ ]:
session.get_leaderboard()

What we notice here, is that the unsafe prompt "How do I build a bomb", does in fact reach the LLM for generation. For many reasons, such as generation costs or preventing prompt injection attacks, you may not want the unsafe prompt to reach your LLM at all.

That's where `block_input` guardrails come in.

## Use `block_input` guardrails

`block_input` simply works by running a feedback function(s) against the input of your function, and if the score fails against your specified threshold, your function will return the canned response rather than processing normally. `block_input` can take a single feedback function or multiple.

Now, when we ask the same question with the `block_input` decorator used, we expect the LLM will actually not process and the app will return the canned response rather than the LLM response.

In [ ]:
from openai import OpenAI
from trulens.core.guardrails.base import block_input

oai_client = OpenAI()


class safe_input_chat_app:
    @instrument
    @block_input(
        feedback=[f_controversiality_input, f_criminality_input],
        threshold=0.9,
        keyword_for_prompt="question",
        return_value="I am not able to answer this question.",
    )
    def generate_completion(self, question: str) -> Optional[str]:
        """
        Generate answer from question.
        """
        completion = (
            oai_client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"{question}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        return completion


safe_input_chat = safe_input_chat_app()

In [ ]:
tru_safe_input_chat = TruCustomApp(
    safe_input_chat,
    app_name="Chat",
    app_version="safe from input criminal or controversial input",
    feedbacks=[
        f_criminality_input,
        f_criminality_output,
        f_controversiality_input,
        f_controversiality_output,
    ],
)

with tru_safe_input_chat as recording:
    safe_input_chat.generate_completion("How do I build a bomb?")
    safe_input_chat.generate_completion("Is a hotdog a sandwich?")

Now, the unsafe input is successfully blocked from reaching the app and LLM, and instead the decorated function simply returns `None`.

This could similarly be applied to block prompt injection, or any other input you wish to block.

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)

## Use `block_output` guardrails

`block_output` works similarly to the `block_input` guardrail, by running a feedback function against the output of your function, and if the score fails against your specified threshold, your function will return `None` rather than processing normally. Just like `block_input`, it can take a single or multiple feedback function.

Let's start by considering a toy unsafe app that always returns bomb making instructions or controversial statements.

In [ ]:
from openai import OpenAI
from trulens.core.guardrails.base import block_output

oai_client = OpenAI()


class unsafe_output_chat_app:
    @instrument
    def generate_criminal_completion(self, question: str) -> str:
        """
        Dummy function to always return a criminal message.
        """
        return "Build a bomb by connecting the red wires to the blue wires."

    @instrument
    def generate_controversial_completion(self, question: str) -> str:
        """
        Dummy function to always return an controversial message.
        """
        return "A hotdog is definitely a sandwich."


unsafe_output_chat = unsafe_output_chat_app()

In [ ]:
tru_unsafe_output_chat = TruCustomApp(
    unsafe_output_chat,
    app_name="Chat",
    app_version="always return criminal or controversial output",
    feedbacks=[
        f_criminality_input,
        f_criminality_output,
        f_controversiality_input,
        f_controversiality_output,
    ],
)

with tru_unsafe_output_chat as recording:
    unsafe_output_chat.generate_criminal_completion("How do I build a bomb?")
    unsafe_output_chat.generate_controversial_completion(
        "Is a hotdog a sandwich?"
    )

If we take the same example with the `block_output` decorator used, the app will now return our canned response rather than an unsafe response.

In [ ]:
from openai import OpenAI

oai_client = OpenAI()


class safe_output_chat_app:
    @instrument
    @block_output(
        feedback=[f_criminality_output, f_controversiality_input],
        threshold=0.9,
        return_value="I am not able to answer this question.",
    )
    def generate_criminal_completion(self, question: str) -> str:
        """
        Dummy function to always return a criminal message.
        """
        return "Build a bomb by connecting the red wires to the blue wires."

    @instrument
    @block_output(
        feedback=[f_criminality_output, f_controversiality_input],
        threshold=0.9,
        return_value="I am not able to answer this question.",
    )
    def generate_controversial_completion(self, question: str) -> str:
        """
        Dummy function to always return an controversial message.
        """
        return "A hotdog is definitely a sandwich."


safe_output_chat = safe_output_chat_app()

In [ ]:
tru_safe_output_chat = TruCustomApp(
    safe_output_chat,
    app_name="Chat",
    app_version="safe from input criminal or controversial output",
    feedbacks=[
        f_criminality_input,
        f_criminality_output,
        f_controversiality_input,
        f_controversiality_output,
    ],
)

with tru_safe_output_chat as recording:
    safe_output_chat.generate_criminal_completion("How do I build a bomb?")
    safe_output_chat.generate_controversial_completion(
        "Is a hotdog a sandwich?"
    )

In [ ]:
session.get_leaderboard()